## CS 6241 Homework 2
### Implementing iterated rank-1 tensor decomposition to interpret time-series Beijing air quality data measured from different stations
Yujia Zhang yz685@cornell.edu

Data source: https://archive.ics.uci.edu/ml/datasets/Beijing+Multi-Site+Air-Quality+Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
from sklearn.decomposition import PCA

### 1. Implement rank-1 CP decomposition

In [211]:
# implementing rank-1 CP decomposition:
# define a function (tensor, number of iterations)

def CPdecomp(A, niter):
    m,n,k = A.shape
    x = np.random.rand(m)
    y = np.random.rand(n)
    z = np.random.rand(k)
    
    for iternum in range(niter):
        # first minimize with respect to $x$
        # then update x, and minimize with respect to y
        # then update y, and minimize with respect to z
    
        for i in range(m):
            a = A[i,:,:].flatten()
            b = np.outer(y,z).flatten()
            x[i] = np.dot(a,b)/np.dot(b,b)
        
        for j in range(n):
            a = A[:,j,:].flatten()
            b = np.outer(x,z).flatten()
            y[j] = np.dot(a,b)/np.dot(b,b)
            
        for l in range(k):
            a = A[:,:,l].flatten()
            b = np.outer(x,y).flatten()
            z[l] = np.dot(a,b)/np.dot(b,b)
    
    return x,y,z

In [ ]:
# TODO: create a random tensor, test the method, and illustrate convergence and error

a=np.array([1,2])
b=np.array([3,4])
c=np.outer(a,b)
d=np.array([5,6,7])
slices=[]
for i in range(len(d)):
    slices.append(c*d[i])

cd=slices[0]
for i in range(1,len(d)):
    cd=np.array([cd,slices[i]])

In [229]:
# run it on a randomly generated small tensor
Atest=np.random.rand(60).reshape(4,3,5)
a,b,c=CPdecomp(Atest,500)
print(a)
print(b)
print(c)

[2.60537108 3.99709405 3.17387092 2.80858305]
[0.27709338 0.25628853 0.2641517 ]
[0.59698531 0.58897538 0.56274477 0.63053578 0.48036481]


In [231]:
# explaining power of the rank-1 decomposition: not so good.
# In practice we can try iterating the rank-1 approximation on the residual at each step
# which is what we will do in the following

ab=np.outer(a,b)
reconstructed=np.outer(ab,c).reshape(4,3,5)
np.linalg.norm(Atest-reconstructed)/np.linalg.norm(Atest)

[[0.72193108 0.66772671 0.68821321]
 [1.10756831 1.02440935 1.0558392 ]
 [0.87945863 0.8134267  0.83838341]
 [0.77823978 0.71980761 0.741892  ]]


0.47637592057550737

### 2. Data analysis

In [310]:
# import data from 12 stations and form 3rd-order tensor

stationnames = ['Aotizhongxin', 'Changping', 'Dingling', 'Dongsi', 'Guanyuan',\
                'Gucheng', 'Huairou', 'Nongzhanguan', 'Shunyi', 'Tiantan',\
               'Wanliu', 'Wanshouxigong']
datacols = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'WSPM']

A=[]
for station in stationnames:
    data=pd.read_csv("/Users/yujiazhang/Desktop/CS 6241/Homework 2/AirData/"+station+".csv")
    B=data[datacols].values
    A.append(B)

C=np.array([A[0],A[1],A[2],A[3],A[4],A[5],A[6],A[7],A[8],A[9],A[10],A[11]]) 

In [303]:
# What the dataframe actually looks like:
data.head(10)
data.shape

# We take the numerical values from
# columns ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'WSPM']

(35064, 18)

In [258]:
C.shape

(12, 35064, 10)

In [261]:
# Clean up the data: brutally drop all slices with NaN values
# This problem might be attacked with matrix completion methods in the future?

D=C[:,range(35064),:]
print('shape of data array before cleaning: '+str(D.shape))
droplist = []
for j in range(D.shape[1]):
    if sum(np.isnan(D[:,j,:]).any(axis=1)) != 0:
        droplist.append(j)

print('number of invalid data rows dropped: '+str(len(droplist)))
valid = np.delete(range(35064), droplist)

D=D[:,valid,:]
print('shape of data array after dropping NaN values: '+str(D.shape))

shape of data array before cleaning: (12, 35064, 10)
number of invalid data rows dropped: 17946
shape of data array after dropping NaN values: (12, 17118, 10)


In [262]:
# Pre-process the data: normalize to center around mean
# because each of the 10 metrics are of different scales, we normalize each [:,:,l] slice

m,n=D[:,:,0].shape

for l in range(D.shape[2]):
    D[:,:,l] = np.matmul( (np.identity(m) - np.ones((m, m)) / m), D[:,:,l])
    
    # normalizing the variance causes NaN values. I haven't figured out why,
    # but for now let's just not do it.
    
    #for j in range(D.shape[1]):
        #D[:,j,l] = D[:,j,l] / np.linalg.norm(D[:,j,l])


In [264]:
# Compute first-order approximation

xD,yD,zD = CPdecomp(D,100)

print(xD)
print(yD)
print(zD)

In [268]:
# first-order error rate is huge
r1approx=np.outer(np.outer(xD,yD),zD).reshape(m,n,10)
np.linalg.norm(D-r1approx)/np.linalg.norm(D)

0.8045828650784138

In [272]:
# compute second-order approximation. Error rate decreases by ~10%
xD2, yD2, zD2 = CPdecomp(D-r1approx, 100)
r2approx = np.outer(np.outer(xD2, yD2), zD2).reshape(m,n,10)
np.linalg.norm(D-r1approx-r2approx)/np.linalg.norm(D)

0.7060123956552663

In [271]:
# Compute third-order approximation. Error rate decreases by ~8%
xD3, yD3, zD3 = CPdecomp(D-r1approx-r2approx, 100)
r3approx = np.outer(np.outer(xD3, yD3), zD3).reshape(m,n,10)
np.linalg.norm(D-r1approx-r2approx-r3approx)/np.linalg.norm(D)

0.6283587427024288

In [273]:
# Compute fourth-order approximation. Error rate decreases by ~7%
xD4, yD4, zD4 = CPdecomp(D-r1approx-r2approx-r3approx, 100)
r4approx = np.outer(np.outer(xD4, yD4), zD4).reshape(m,n,10)
np.linalg.norm(D-r1approx-r2approx-r3approx-r4approx)/np.linalg.norm(D)

0.5596077240231145

In [274]:
# Compute fifth-order approximation. Error rate decreases by ~6%
xD5, yD5, zD5 = CPdecomp(D-r1approx-r2approx-r3approx-r4approx, 100)
r5approx = np.outer(np.outer(xD5, yD5), zD5).reshape(m,n,10)
np.linalg.norm(D-r1approx-r2approx-r3approx-r4approx-r5approx)/np.linalg.norm(D)

0.49863287138163437

In [275]:
# Compute sixth-order approximation. Error rate decreases by ~6%
xD6, yD6, zD6 = CPdecomp(D-r1approx-r2approx-r3approx-r4approx-r5approx, 100)
r6approx = np.outer(np.outer(xD6, yD6), zD6).reshape(m,n,10)
np.linalg.norm(D-r1approx-r2approx-r3approx-r4approx-r5approx-r6approx)/np.linalg.norm(D)

0.43870000258546044

In [276]:
# Compute seventh-order approximation. Error rate decreases by ~6%
xD7, yD7, zD7 = CPdecomp(D-r1approx-r2approx-r3approx-r4approx-r5approx-r6approx, 100)
r7approx = np.outer(np.outer(xD7, yD7), zD7).reshape(m,n,10)
np.linalg.norm(D-r1approx-r2approx-r3approx-r4approx-r5approx-r6approx-r7approx)/np.linalg.norm(D)

0.3781907284173674

In [304]:
# Compute eighth-order approximation. Error rate decreases by ~6%
xD8, yD8, zD8 = CPdecomp(D-r1approx-r2approx-r3approx-r4approx-r5approx-r6approx-r7approx, 100)
r8approx = np.outer(np.outer(xD8, yD8), zD8).reshape(m,n,10)
np.linalg.norm(D-r1approx-r2approx-r3approx-r4approx-r5approx-r6approx-r7approx-r8approx)/np.linalg.norm(D)

0.3179015670765071

In [305]:
# Compute ninth-order approximation. Error rate decreases by ~7%
xD9, yD9, zD9 = CPdecomp(D-r1approx-r2approx-r3approx-r4approx-r5approx-r6approx-r7approx-r8approx, 100)
r9approx = np.outer(np.outer(xD9, yD9), zD9).reshape(m,n,10)
np.linalg.norm(D-r1approx-r2approx-r3approx-r4approx-r5approx-r6approx-r7approx-r8approx-r9approx)/np.linalg.norm(D)

0.2479413102266508

In [307]:
# Compute tenth-order approximation. Error rate decreases by ~6%
xD10, yD10, zD10 = CPdecomp(D-r1approx-r2approx-r3approx-r4approx-r5approx-r6approx-r7approx-r8approx-r9approx, 100)
r10approx = np.outer(np.outer(xD10, yD10), zD10).reshape(m,n,10)
np.linalg.norm(D-r1approx-r2approx-r3approx-r4approx-r5approx-r6approx-r7approx-r8approx-r9approx-r10approx)/np.linalg.norm(D)

0.18252055157010888

### 3. Results and interpretation

Now let us try to intepret the factors qualitatively. Recall that $x\in\mathbb{R}^{12}$ corresponds to the twelve stations at which measurements were taken, and $z\in\mathbb{R}^{10}$ corresponds to the ten metrics for air quality.

In [309]:
# Let's look at the top 5 x-factors and try to interpret them
# Recall: x is 12-dimensional, corresponding to the 12 stations

print(np.c_[xD.reshape(len(xD),1), xD2.reshape(len(xD),1), xD3.reshape(len(xD),1), xD4.reshape(len(xD),1),\
           xD5.reshape(len(xD),1)])


[[ 30.52604223  -0.82265533   1.30872902 -11.85116029 -16.60384968]
 [-49.49299978  44.67120413  -0.238763   -11.06995116  19.33304633]
 [-83.56880777  11.8830623   -8.73379534  -0.52723091 -13.03601823]
 [ 39.27508733 -12.04687417  -5.94851484   0.15686066   4.21206407]
 [ 24.19615187   3.92645797  -2.80933181   2.22256751  -5.20826871]
 [  4.87205552  24.03180142   8.5397304   14.31708718  -2.98689033]
 [-75.24318155 -24.94525532  -0.25830754   8.47918408  -2.32512246]
 [ 44.87436845 -13.02470072  -3.50256444  -7.97327105  -1.43734839]
 [-30.76052355 -55.92034969  10.83761846  -6.19459726   6.7394871 ]
 [ 31.06848019  -6.71721856  -5.82691661   5.41534848   7.67244142]
 [ 25.95235996  34.12667502  10.15571459  -0.66726431  -5.3198347 ]
 [ 38.30096709  -5.16214704  -3.5235989    7.69242707   8.96029357]]


In [302]:
# And the top 4 z-factors
# Recall: z is 10-dimensional, corresponding to the 10 air quality metrics

print(np.c_[zD.reshape(len(zD),1), zD2.reshape(len(zD),1), zD3.reshape(len(zD),1), zD4.reshape(len(zD),1)])


[[ 2.13721292e-01  1.10139465e-01  3.55230376e-01  1.96462026e-01]
 [ 2.61068127e-01  1.29004968e-01  4.68613905e-01  2.44178836e-01]
 [ 4.08322450e-02  2.15398199e-02  6.39513243e-02  3.55188581e-02]
 [ 1.45770557e-01  4.15394352e-02  2.09181012e-01  1.00650894e-01]
 [ 4.59606642e+00  2.98846310e+00  1.26829437e+01  6.68225919e+00]
 [-3.95095161e-02 -1.73207678e-02 -8.73495635e-02 -2.42272170e-02]
 [ 1.91165806e-04 -2.94027005e-04 -4.69651753e-03 -1.07998298e-03]
 [ 1.10048904e-02 -7.40783211e-04  8.15779987e-04 -2.30845342e-04]
 [ 5.67604042e-03  1.92419494e-03  7.21862620e-03  1.97160789e-03]
 [-6.99720119e-04 -3.02006360e-04 -2.11901158e-03 -7.04764190e-04]]


### 4. Future work

1. Use matrix completion to deal with missing data.
2. Take into account the relative geographical locations of the 12 stations. Maybe look at pairwise distance between the stations, and also account for altitude, population density, traffic flow density, etc.
3. Normalize the data with respect to its standard deviation. 
4. Constrain to nonnegative factors